# Basic Network Notebook

This notebook contains the definition for a basic network that will 
play tic tac toe. This notebook also contains code to train this 
network utilizing Q learning


In [4]:
# Import Tic Tac Toe code
from game import TicTacToe

# Make a game
toe = TicTacToe()

# Define Network

Below is the code to create a nerual network to play the 
tic tac toe game. 

In [5]:
import tensorflow as tf

# Define settings
board_size = toe.size * toe.size

# Trying for simple DNN right now
hidden = 9

# Define output space size, one output for each space
# Network will output the space it wants to play on
output_size = board_size

# Learning Rate
learning_rate = 0.5

board = tf.placeholder(dtype=tf.int8, 
                       shape=[board_size],
                       name="Board")

#Weights
W = tf.Variable(tf.random_normal(shape=[hidden, output_size]),
                name="Weights")
#Biases
b = tf.Variable(tf.zeros(shape=[output_size]),
                name="Biases")

#Output
qout = tf.matmul(inputs, W) + b

#Prediction
pred = tf.argmax(qout)


